In [15]:
import pandas as pd
import numpy as np
import ast
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
movies=pd.read_csv("tmdb_5000_movies.csv")
data=pd.read_csv("tmdb_5000_credits.csv")

In [17]:
movies=movies.merge(data,on='title')
movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [18]:
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name'])
    return L
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [19]:
def convert3(text):
    L = []
    count=0
    for i in ast.literal_eval(text):
        if count != 3:
            L.append(i['name'])
            count+=1
        else:
            break
    return L
movies['cast'] = movies['cast'].apply(convert3)

In [20]:
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L
movies['crew'] = movies['crew'].apply(fetch_director)
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [21]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

movies['overview'] = movies['overview'].apply(lambda x: str(x).split())
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']
new = movies.drop(columns=['overview','genres','keywords','cast','crew'])
new['tags'] = new['tags'].apply(lambda x: " ".join(x))
new['tags']=new['tags'].apply(lambda x:x.lower())

In [22]:
ps=PorterStemmer()

def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))

    return " ".join(y)

new['tags']=new['tags'].apply(stem)
cv = CountVectorizer(max_features=5000,stop_words='english')
vectors=cv.fit_transform(new['tags']).toarray()
similarity=cosine_similarity(vectors)

sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]


# this is the main function which will be used for prediction by using streamlit library in pychram
def recommend(movie):
    movie_index=new[new['title']==movie].index[0]
    distances=similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in movies_list:
        print(new.iloc[i[0]].title)


In [23]:
recommend('Avatar')

Aliens vs Predator: Requiem
Aliens
Falcon Rising
Independence Day
Titan A.E.


In [24]:
import joblib
joblib.dump(similarity, 'similarity.joblib', compress=3)

['similarity.joblib']

In [25]:
# Extra code used for dumping the data into pycharm using pickle
import pickle
pickle.dump(new,open('movies.pkl','wb'))  # name the file by which you want to save
pickle.dump(new.to_dict(),open('movie_dict.pkl','wb'))